## Counter

In [1]:
import magma as m
m.set_mantle_target("ice40")

## Circuit 

To implement a counter we can use the `Register` primitive and the `+` operator. Arithmetic operators are available on the types `UInt` and `SInt`, but not on `Bits`.

In this example we show how to construct a new `Circuit`. 
A circuit in `magma` is like a function in a programming language.

A circuit is also a verilog module. 
To use a circuit, we need to create an instance.
Once we have an instance, we can wire up its inputs and outputs.

In [2]:
from mantle import Register

class SimpleCounter4(m.Circuit):
    IO = ["O", m.Out(m.UInt(4))] + m.ClockInterface()
    @classmethod
    def definition(io):
        reg4 = Register(4)
        m.wire(reg4(m.uint(reg4.O) + m.uint(1,4)), io.O)

import lattice ice40
import lattice mantle40


First, notice that we subclass `m.Circuit` to create a new circuit.
The name of the circuit will be the name of the subclass.
A circuit also has an interface.
We declare the interface by setting the class attribute `IO`.
This is just a list of names and types.
Using lists has the advantage that we can build constructors
that build interfaces by creating different lists.
In this example, we add the default
clock interface to the interface.

Inside the class is a `classmethod` named `definition`. 
This method is called to construct the actual circuit.
The interface variables are passed in as a Python object
containing named instances of the IO variables. 
In this case, the only visible interface variable is `O`.

This counter is very simple. 
We create a register,
and wire the input of the register to 
the output of the register plus one.

Finally, note that we wire the output of the register
to the counter output `O`.

## Counter Generator

The above example creates a counter of size 4. How can we make a more general counter of arbitray width?

To do this we create a Python generator.
Generators are functions with parameters
that return a `Circuit` class.

In `magma` we use the convention that
generators begin with the prefix `Define`.

In [3]:
def DefineCounter(n):
    class _Counter(m.Circuit):
        name = 'Counter{}'.format(n)
        IO = ["O", m.Out(m.UInt(n))] + m.ClockInterface()
        @classmethod
        def definition(io):
            reg = Register(n)
            m.wire(reg(m.uint(reg.O) + m.uint(1,n)), io.O)
    return _Counter

def Counter(n):
    return DefineCounter(n)()

In [4]:
from loam.boards.icestick import IceStick

N = 30

icestick = IceStick()
icestick.Clock.on()
for i in range(8):
    icestick.J3[i].output().on()
    
main = icestick.DefineMain()

counter = Counter(N)
m.wire( counter.O[N-8:N], main.J3 )

m.EndDefine()

## Compile and Build

We can compile and inspect the verilog.

In [5]:
m.compile("build/counter", main)

compiling Register30
compiling FullAdder
compiling Add30
compiling Counter30
compiling main


In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif counter.blif' counter.v
arachne-pnr -q -d 1k -o counter.txt -p counter.pcf counter.blif 
icepack counter.txt counter.bin
iceprog counter.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x12 0x67 0x21 0x20 0x00 0x61 0x00 0x53 0x04 0x11 0x11 0x03 0xB1
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
